## 임베딩을 사용한 코드 검색

이 노트북에서는 Ada 임베딩을 사용하여 시맨틱 코드 검색을 구현하는 방법을 보여드립니다. 이 데모에서는 자체 [openai-python 코드 리포지토리](https://github.com/openai/openai-python)를 사용합니다. 임베딩, 인덱싱 및 쿼리할 수 있는 간단한 버전의 파일 구문 분석 및 파이썬 파일에서 함수 추출을 구현합니다.

### 도우미 기능

먼저 코드베이스에서 중요한 정보를 추출할 수 있는 몇 가지 간단한 구문 분석 함수를 설정했습니다.

In [ ]:
import pandas as pd
from pathlib import Path

DEF_PREFIXES = ['def ', 'async def ']
NEWLINE = '\n'

def get_function_name(code):
    """
    Extract function name from a line beginning with 'def' or 'async def'.
    """
    for prefix in DEF_PREFIXES:
        if code.startswith(prefix):
            return code[len(prefix): code.index('(')]


def get_until_no_space(all_lines, i):
    """
    Get all lines until a line outside the function definition is found.
    """
    ret = [all_lines[i]]
    for j in range(i + 1, len(all_lines)):
        if len(all_lines[j]) == 0 or all_lines[j][0] in [' ', '\t', ')']:
            ret.append(all_lines[j])
        else:
            break
    return NEWLINE.join(ret)


def get_functions(filepath):
    """
    Get all functions in a Python file.
    """
    with open(filepath, 'r') as file:
        all_lines = file.read().replace('\r', NEWLINE).split(NEWLINE)
        for i, l in enumerate(all_lines):
            for prefix in DEF_PREFIXES:
                if l.startswith(prefix):
                    code = get_until_no_space(all_lines, i)
                    function_name = get_function_name(code)
                    yield {
                        'code': code,
                        'function_name': function_name,
                        'filepath': filepath,
                    }
                    break


def extract_functions_from_repo(code_root):
    """
    Extract all .py functions from the repository.
    """
    code_files = list(code_root.glob('**/*.py'))

    num_files = len(code_files)
    print(f'Total number of .py files: {num_files}')

    if num_files == 0:
        print('Verify openai-python repo exists and code_root is set correctly.')
        return None

    all_funcs = [
        func
        for code_file in code_files
        for func in get_functions(str(code_file))
    ]

    num_funcs = len(all_funcs)
    print(f'Total number of functions extracted: {num_funcs}')

    return all_funcs

# 데이터 로드

먼저 openai-python 폴더를 로드하고 위에서 정의한 함수를 사용하여 필요한 정보를 추출하겠습니다.

In [10]:
# Set user root directory to the 'openai-python' repository
root_dir = Path.home()

# Assumes the 'openai-python' repository exists in the user's root directory
code_root = root_dir / 'openai-python'

# Extract all functions from the repository
all_funcs = extract_functions_from_repo(code_root)

Total number of .py files: 57
Total number of functions extracted: 118


이제 콘텐츠를 확보했으므로 데이터를 텍스트 임베딩-ada-002 엔드포인트로 전달하여 벡터 임베딩을 반환할 수 있습니다.

In [11]:
from openai.embeddings_utils import get_embedding

df = pd.DataFrame(all_funcs)
df['code_embedding'] = df['code'].apply(lambda x: get_embedding(x, engine='text-embedding-ada-002'))
df['filepath'] = df['filepath'].map(lambda x: Path(x).relative_to(code_root))
df.to_csv("data/code_search_openai-python.csv", index=False)
df.head()

,code,function_name,filepath,code_embedding
0,def _console_log_level():\n if openai.log i...,_console_log_level,openai/util.py,"[0.033906757831573486, -0.00418944051489234, 0..."
1,"def log_debug(message, **params):\n msg = l...",log_debug,openai/util.py,"[-0.004059609025716782, 0.004895503632724285, ..."
2,"def log_info(message, **params):\n msg = lo...",log_info,openai/util.py,"[0.0048639848828315735, 0.0033139237202703953,..."
3,"def log_warn(message, **params):\n msg = lo...",log_warn,openai/util.py,"[0.0024026145692914724, -0.010721310041844845,..."
4,"def logfmt(props):\n def fmt(key, val):\n ...",logfmt,openai/util.py,"[0.01664826273918152, 0.01730910874903202, 0.0..."


### 테스트

몇 가지 간단한 쿼리로 엔드포인트를 테스트해 봅시다. 'openai-python' 리포지토리에 익숙하다면 간단한 영어 설명만으로 원하는 함수를 쉽게 찾을 수 있다는 것을 알 수 있습니다.

임베딩, 쿼리 문자열 및 기타 구성 옵션이 포함된 데이터를 가져오는 search_functions 메서드를 정의합니다. 데이터베이스 검색 프로세스는 이와 같이 작동합니다:

1. 먼저 텍스트 임베딩-ada-002로 쿼리 문자열(code_query)을 임베딩합니다. 그 이유는 '문자열을 반전시키는 함수'와 같은 쿼리 문자열과 'def reverse(문자열): 반환 문자열[::-1]'과 같은 함수가 임베드될 때 매우 유사하기 때문입니다.
2. 그런 다음 쿼리 문자열 임베딩과 데이터베이스의 모든 데이터 포인트 사이의 코사인 유사도를 계산합니다. 이렇게 하면 각 포인트와 쿼리 사이의 거리가 계산됩니다.
3. 마지막으로 모든 데이터 포인트를 쿼리 문자열과의 거리에 따라 정렬하고 함수 매개변수에 요청된 결과 수를 반환합니다.

In [1]:
from openai.embeddings_utils import cosine_similarity

def search_functions(df, code_query, n=3, pprint=True, n_lines=7):
    embedding = get_embedding(code_query, engine='text-embedding-ada-002')
    df['similarities'] = df.code_embedding.apply(lambda x: cosine_similarity(x, embedding))

    res = df.sort_values('similarities', ascending=False).head(n)

    if pprint:
        for r in res.iterrows():
            print(f"{r[1].filepath}:{r[1].function_name}  score={round(r[1].similarities, 3)}")
            print("\n".join(r[1].code.split("\n")[:n_lines]))
            print('-' * 70)

    return res

In [13]:
res = search_functions(df, 'fine-tuning input data validation logic', n=3)

openai/validators.py:format_inferrer_validator  score=0.751
def format_inferrer_validator(df):
    """
    This validator will infer the likely fine-tuning format of the data, and display it to the user if it is classification.
    It will also suggest to use ada and explain train/validation split benefits.
    """
    ft_type = infer_task_type(df)
    immediate_msg = None
----------------------------------------------------------------------
openai/validators.py:get_validators  score=0.748
def get_validators():
    return [
        num_examples_validator,
        lambda x: necessary_column_validator(x, "prompt"),
        lambda x: necessary_column_validator(x, "completion"),
        additional_column_validator,
        non_empty_field_validator,
----------------------------------------------------------------------
openai/validators.py:infer_task_type  score=0.739
def infer_task_type(df):
    """
    Infer the likely fine-tuning task type from the data
    """
    CLASSIFICATION_THRES

In [14]:
res = search_functions(df, 'find common suffix', n=2, n_lines=10)

openai/validators.py:get_common_xfix  score=0.794
def get_common_xfix(series, xfix="suffix"):
    """
    Finds the longest common suffix or prefix of all the values in a series
    """
    common_xfix = ""
    while True:
        common_xfixes = (
            series.str[-(len(common_xfix) + 1) :]
            if xfix == "suffix"
            else series.str[: len(common_xfix) + 1]
----------------------------------------------------------------------
openai/validators.py:common_completion_suffix_validator  score=0.778
def common_completion_suffix_validator(df):
    """
    This validator will suggest to add a common suffix to the completion if one doesn't already exist in case of classification or conditional generation.
    """
    error_msg = None
    immediate_msg = None
    optional_msg = None
    optional_fn = None

    ft_type = infer_task_type(df)
----------------------------------------------------------------------


In [15]:
res = search_functions(df, 'Command line interface for fine-tuning', n=1, n_lines=20)

openai/cli.py:tools_register  score=0.78
def tools_register(parser):
    subparsers = parser.add_subparsers(
        title="Tools", help="Convenience client side tools"
    )

    def help(args):
        parser.print_help()

    parser.set_defaults(func=help)

    sub = subparsers.add_parser("fine_tunes.prepare_data")
    sub.add_argument(
        "-f",
        "--file",
        required=True,
        help="JSONL, JSON, CSV, TSV, TXT or XLSX file containing prompt-completion examples to be analyzed."
        "This should be the local file path.",
    )
    sub.add_argument(
        "-q",
----------------------------------------------------------------------
